In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import minmax_scale

"""
Initializes an empty ensemble
"""
def init_hillclimb():
    best_ensemble = {}
    for label in LABELS:
        best_ensemble[label] = []
    best_score = {}
    for label in LABELS:
        best_score[label] = 0
    
    return best_ensemble, best_score

"""
Scores average AUC for an ensemble per label
"""
def score_ensemble(ensemble, label):
    blend_preds = np.zeros(len(train))
    
    for model in ensemble:
        blend_preds += minmax_scale(files[model][label])
        
    blend_preds = blend_preds/len(subnums)
    score = roc_auc_score(train[label], blend_preds)
    
    return score

"""
Finds the optimal model to add next to an ensemble
"""
def find_best_improvement(ensemble, label):
    best_score = 0
    best_ensemble = []
    
    for i in range(0,len(files)):
        ensemble = ensemble + [i]

        score = score_ensemble(ensemble, label)
        
        if score > best_score:
            best_score  = score
            best_ensemble = ensemble
            
        ensemble = ensemble[:-1]
    
    return best_ensemble, best_score
        
"""
Performs a step for each label
"""
def climb(best_ensemble, best_score):
    for label in LABELS:        
        best_ensemble[label], best_score[label] = find_best_improvement(best_ensemble[label], label)
        
    return best_ensemble, best_score

"""
Gets optimal blending weights after hillclimb
"""
def get_optimal_weights(best_ensemble):
    weights = {}
    for label in LABELS:
        weights[label] = {}
        for num in set(best_ensemble[label]):
            print num
            weights[label][num] = best_ensemble[label].count(num)/float(len(best_ensemble[label]))
    return weights

"""
Constructs a pandas dataframe using the optimal blending weights
"""
def get_optimal_blend(optimal_weights):
    sub = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))
    blend = sub.copy()
    for label in LABELS:
        print(label)
        for key in optimal_weights[label]:
            blend[label] += optimal_weights[label][key] * minmax_scale(get_sub_file(key)[label])
            print(optimal_weights[label][key], filenames[key])
            blend[label] = minmax_scale(blend[label])
        
    return blend

def get_sub_file(num):
    SUB_PATH = "SUB"
    filename = filenames[num]
    filename = filename.replace("oof", "sub")
    return pd.read_csv(os.path.join(SUB_PATH, filename))


if __name__ == "__main__":
    DATA_PATH = ""
    SUB_PATH = "OOF"
    
    train = pd.read_csv(os.path.join(DATA_PATH, "train.csv")).fillna(' ')
    test = pd.read_csv(os.path.join(DATA_PATH, "test.csv")).fillna(' ')
    
    LABELS = train.columns[2:]
    
    # Get submissions and out-of-fold predictions
    subnums = [1,2,3,4,5,6,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
    filenames = ["oof" + str(num) + ".csv" for num in subnums]
    files = [pd.read_csv(os.path.join(SUB_PATH, filename)) for filename in filenames]
    
    best_ensemble, best_score = init_hillclimb()
    
    # Run hillclimb
    for i in range(200):
        print("-------------")
        print("Step", i)    
        best_ensemble, best_score = climb(best_ensemble, best_score)
        print("Best ensemble:")
        print(best_ensemble)
        print("Best score:")
        print(best_score)
        print("AUC:", np.mean([best_score[label] for label in LABELS]))
    
    # Get optimal weights
    opt_w = get_optimal_weights(best_ensemble)
    print("Optimal weights:")
    print(opt_w)
    
    # Construct the blend
    blend = get_optimal_blend(opt_w)
    
    # Submit
    blend.to_csv("hillclimb.csv", index=False)

-------------
('Step', 0)
Best ensemble:
{'severe_toxic': [8], 'identity_hate': [8], 'obscene': [8], 'insult': [8], 'threat': [1], 'toxic': [8]}
Best score:
{'severe_toxic': 0.990704317929012, 'identity_hate': 0.9865429685276377, 'obscene': 0.9930640355099863, 'insult': 0.988151944616848, 'threat': 0.9913979684049437, 'toxic': 0.9846902019085222}
('AUC:', 0.9890919061494916)
-------------
('Step', 1)
Best ensemble:
{'severe_toxic': [8, 2], 'identity_hate': [8, 6], 'obscene': [8, 25], 'insult': [8, 7], 'threat': [1, 25], 'toxic': [8, 25]}
Best score:
{'severe_toxic': 0.9916022679053029, 'identity_hate': 0.9888896088856238, 'obscene': 0.9946895156920439, 'insult': 0.9891569227687192, 'threat': 0.9926745894029456, 'toxic': 0.9862616835151461}
('AUC:', 0.9905457646949637)
-------------
('Step', 2)
Best ensemble:
{'severe_toxic': [8, 2, 25], 'identity_hate': [8, 6, 2], 'obscene': [8, 25, 2], 'insult': [8, 7, 25], 'threat': [1, 25, 1], 'toxic': [8, 25, 1]}
Best score:
{'severe_toxic': 0.9918

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6], 'obscene': [8, 25, 2, 7, 1, 25, 6, 8, 25, 1, 25, 7, 5, 8, 2, 25, 1], 'insult': [8, 7, 25, 1, 2, 7, 8, 1, 25, 7, 8, 1, 25, 7, 8, 1, 4], 'threat': [1, 25, 1, 3, 2, 1, 7, 1, 5, 8, 1, 6, 1, 2, 5, 1, 3], 'toxic': [8, 25, 1, 7, 25, 2, 8, 25, 1, 25, 7, 8, 5, 1, 25, 2, 25]}
Best score:
{'severe_toxic': 0.9920482108071492, 'identity_hate': 0.9912791115492292, 'obscene': 0.9951692738481855, 'insult': 0.98977770772933, 'threat': 0.9940139352191227, 'toxic': 0.9875597430987215}
('AUC:', 0.9916413303752898)
-------------
('Step', 17)
Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2], 'obscene': [8, 25, 2, 7, 1, 25, 6, 8, 25, 1, 25, 7, 5, 8, 2, 25, 1, 25], 'insult': [8, 7, 25, 1, 2, 7, 8, 1, 25, 7, 8, 1, 25, 7, 8, 1, 4, 25], 'threat': 

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8], 'obscene': [8, 25, 2, 7, 1, 25, 6, 8, 25, 1, 25, 7, 5, 8, 2, 25, 1, 25, 4, 7, 8, 25, 1, 25, 25, 8, 2, 7], 'insult': [8, 7, 25, 1, 2, 7, 8, 1, 25, 7, 8, 1, 25, 7, 8, 1, 4, 25, 5, 1, 8, 7, 25, 8, 1, 7, 25, 2], 'threat': [1, 25, 1, 3, 2, 1, 7, 1, 5, 8, 1, 6, 1, 2, 5, 1, 3, 1, 8, 5, 7, 1, 2, 5, 1, 7, 2, 5], 'toxic': [8, 25, 1, 7, 25, 2, 8, 25, 1, 25, 7, 8, 5, 1, 25, 2, 25, 8, 7, 1, 4, 25, 25, 1, 8, 7, 25, 0]}
Best score:
{'severe_toxic': 0.992049302199469, 'identity_hate': 0.991320776860277, 'obscene': 0.9951907904203031, 'insult': 0.9898102809214221, 'threat': 0.9940504523721777, 'toxic': 0.9876135364834101}
('AUC:', 0.9916725232095098)
-------------
('Step', 28)
Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7], 'obscene': [8, 25, 2, 7, 1, 25, 6, 8, 25, 1, 25, 7, 5, 8, 2, 25, 1, 25, 4, 7, 8, 25, 1, 25, 25, 8, 2, 7, 25, 1, 6, 8, 25, 5, 1, 25, 7], 'insult': [8, 7, 25, 1, 2, 7, 8, 1, 25, 7, 8, 1, 25, 7, 8, 1, 4, 25, 5, 1, 8, 7, 25, 8, 1, 7, 25, 2, 8, 7, 1, 11, 25, 1, 8, 7, 6], 'threat': [1, 25, 1, 3, 2, 1, 7, 1, 5, 8, 1, 6, 1, 2, 5, 1, 3, 1, 8, 5, 7, 1, 2, 5, 1, 7, 2, 5, 1, 8, 3, 5, 1, 2, 5, 7, 1], 'toxic': [8, 25, 1, 7, 25, 2, 8, 25, 1, 25, 7, 8, 5, 1, 25, 2, 25, 8, 7, 1, 4, 25, 25, 1, 8, 7, 25, 0, 25, 8, 1, 25, 7, 2, 8, 25, 1]}
Best score:
{'severe_toxic': 0.9920498855982728, 'identity_hate': 0.9913328863053605, 'obscene': 0.995192766408053, 'insult': 0.9898161965107056, 'threat': 0.9940406162790969, 'toxic': 0.98761700

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2, 7, 8, 25, 1, 2, 7, 8, 5], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7, 8, 1, 5, 7, 2, 6, 8, 3], 'obscene': [8, 25, 2, 7, 1, 25, 6, 8, 25, 1, 25, 7, 5, 8, 2, 25, 1, 25, 4, 7, 8, 25, 1, 25, 25, 8, 2, 7, 25, 1, 6, 8, 25, 5, 1, 25, 7, 2, 8, 25, 25, 1, 8, 25, 3], 'insult': [8, 7, 25, 1, 2, 7, 8, 1, 25, 7, 8, 1, 25, 7, 8, 1, 4, 25, 5, 1, 8, 7, 25, 8, 1, 7, 25, 2, 8, 7, 1, 11, 25, 1, 8, 7, 6, 8, 1, 7, 2, 25, 8, 7, 1], 'threat': [1, 25, 1, 3, 2, 1, 7, 1, 5, 8, 1, 6, 1, 2, 5, 1, 3, 1, 8, 5, 7, 1, 2, 5, 1, 7, 2, 5, 1, 8, 3, 5, 1, 2, 5, 7, 1, 8, 5, 3, 2, 1, 25, 1, 1], 'toxic': [8, 25, 1, 7, 25, 2, 8, 25, 1, 25, 7, 8, 5, 1, 25, 2, 25, 8, 7, 1, 4, 25, 25, 1, 8, 7, 25, 0, 25, 8, 1, 25, 7, 2, 8, 25, 1, 25, 5, 8, 7, 1, 25, 3, 25]}
Best score:
{'severe_toxic': 0.99205055233

...

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2, 7, 8, 25, 1, 2, 7, 8, 5, 2, 1, 8, 6, 7, 2, 1, 8, 25, 3, 8, 5, 7, 1, 2, 8, 25, 7, 7, 5, 1, 8, 2, 2, 1, 7, 25, 1, 8, 2, 6, 8, 3, 5, 7, 1, 2, 8, 25, 1, 7, 5, 1, 3, 7, 25, 8, 1, 3, 7, 5, 2, 8, 1, 7, 2, 25, 8, 1, 25, 7, 1, 8, 5, 2, 8, 6, 1, 7, 2, 25, 8, 5, 7, 1, 2, 1, 8, 3, 25, 7, 8, 5, 8, 8, 2, 2, 2, 1, 8, 25, 7, 2, 7, 2, 5, 1, 8, 25, 1, 8, 7, 2, 7, 6, 7, 5, 1, 3, 25, 7, 8, 7, 1, 2, 5, 8, 8, 5, 1, 1, 2, 25, 2, 7, 3, 25, 1, 7, 8, 2, 5], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7, 8, 1, 5, 7, 2, 6, 8, 3, 7, 2, 8, 2, 6, 1, 7, 8, 5, 8, 6, 2, 7, 1, 3, 8, 6, 7, 2, 2, 7, 8, 1, 6, 4, 3, 5, 7, 2, 8, 1, 6, 7, 2, 8, 1, 6, 7, 2, 8, 3, 8, 5, 7, 6, 8, 2, 7, 2, 1, 6, 7, 8, 1, 2, 6, 8, 3, 5, 7, 8, 2, 6, 1, 8, 7, 2, 3, 6, 5, 8, 7, 2, 8, 7, 4, 6, 2, 1, 8, 3, 6, 7, 1, 2, 5, 8, 7

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2, 7, 8, 25, 1, 2, 7, 8, 5, 2, 1, 8, 6, 7, 2, 1, 8, 25, 3, 8, 5, 7, 1, 2, 8, 25, 7, 7, 5, 1, 8, 2, 2, 1, 7, 25, 1, 8, 2, 6, 8, 3, 5, 7, 1, 2, 8, 25, 1, 7, 5, 1, 3, 7, 25, 8, 1, 3, 7, 5, 2, 8, 1, 7, 2, 25, 8, 1, 25, 7, 1, 8, 5, 2, 8, 6, 1, 7, 2, 25, 8, 5, 7, 1, 2, 1, 8, 3, 25, 7, 8, 5, 8, 8, 2, 2, 2, 1, 8, 25, 7, 2, 7, 2, 5, 1, 8, 25, 1, 8, 7, 2, 7, 6, 7, 5, 1, 3, 25, 7, 8, 7, 1, 2, 5, 8, 8, 5, 1, 1, 2, 25, 2, 7, 3, 25, 1, 7, 8, 2, 5, 7, 8, 25], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7, 8, 1, 5, 7, 2, 6, 8, 3, 7, 2, 8, 2, 6, 1, 7, 8, 5, 8, 6, 2, 7, 1, 3, 8, 6, 7, 2, 2, 7, 8, 1, 6, 4, 3, 5, 7, 2, 8, 1, 6, 7, 2, 8, 1, 6, 7, 2, 8, 3, 8, 5, 7, 6, 8, 2, 7, 2, 1, 6, 7, 8, 1, 2, 6, 8, 3, 5, 7, 8, 2, 6, 1, 8, 7, 2, 3, 6, 5, 8, 7, 2, 8, 7, 4, 6, 2, 1, 8, 3, 6, 7, 1, 

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2, 7, 8, 25, 1, 2, 7, 8, 5, 2, 1, 8, 6, 7, 2, 1, 8, 25, 3, 8, 5, 7, 1, 2, 8, 25, 7, 7, 5, 1, 8, 2, 2, 1, 7, 25, 1, 8, 2, 6, 8, 3, 5, 7, 1, 2, 8, 25, 1, 7, 5, 1, 3, 7, 25, 8, 1, 3, 7, 5, 2, 8, 1, 7, 2, 25, 8, 1, 25, 7, 1, 8, 5, 2, 8, 6, 1, 7, 2, 25, 8, 5, 7, 1, 2, 1, 8, 3, 25, 7, 8, 5, 8, 8, 2, 2, 2, 1, 8, 25, 7, 2, 7, 2, 5, 1, 8, 25, 1, 8, 7, 2, 7, 6, 7, 5, 1, 3, 25, 7, 8, 7, 1, 2, 5, 8, 8, 5, 1, 1, 2, 25, 2, 7, 3, 25, 1, 7, 8, 2, 5, 7, 8, 25, 1, 7, 2], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7, 8, 1, 5, 7, 2, 6, 8, 3, 7, 2, 8, 2, 6, 1, 7, 8, 5, 8, 6, 2, 7, 1, 3, 8, 6, 7, 2, 2, 7, 8, 1, 6, 4, 3, 5, 7, 2, 8, 1, 6, 7, 2, 8, 1, 6, 7, 2, 8, 3, 8, 5, 7, 6, 8, 2, 7, 2, 1, 6, 7, 8, 1, 2, 6, 8, 3, 5, 7, 8, 2, 6, 1, 8, 7, 2, 3, 6, 5, 8, 7, 2, 8, 7, 4, 6, 2, 1, 8, 3, 

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2, 7, 8, 25, 1, 2, 7, 8, 5, 2, 1, 8, 6, 7, 2, 1, 8, 25, 3, 8, 5, 7, 1, 2, 8, 25, 7, 7, 5, 1, 8, 2, 2, 1, 7, 25, 1, 8, 2, 6, 8, 3, 5, 7, 1, 2, 8, 25, 1, 7, 5, 1, 3, 7, 25, 8, 1, 3, 7, 5, 2, 8, 1, 7, 2, 25, 8, 1, 25, 7, 1, 8, 5, 2, 8, 6, 1, 7, 2, 25, 8, 5, 7, 1, 2, 1, 8, 3, 25, 7, 8, 5, 8, 8, 2, 2, 2, 1, 8, 25, 7, 2, 7, 2, 5, 1, 8, 25, 1, 8, 7, 2, 7, 6, 7, 5, 1, 3, 25, 7, 8, 7, 1, 2, 5, 8, 8, 5, 1, 1, 2, 25, 2, 7, 3, 25, 1, 7, 8, 2, 5, 7, 8, 25, 1, 7, 2, 1, 8, 5], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7, 8, 1, 5, 7, 2, 6, 8, 3, 7, 2, 8, 2, 6, 1, 7, 8, 5, 8, 6, 2, 7, 1, 3, 8, 6, 7, 2, 2, 7, 8, 1, 6, 4, 3, 5, 7, 2, 8, 1, 6, 7, 2, 8, 1, 6, 7, 2, 8, 3, 8, 5, 7, 6, 8, 2, 7, 2, 1, 6, 7, 8, 1, 2, 6, 8, 3, 5, 7, 8, 2, 6, 1, 8, 7, 2, 3, 6, 5, 8, 7, 2, 8, 7, 4, 6, 2, 

Best ensemble:
{'severe_toxic': [8, 2, 25, 7, 1, 5, 8, 3, 1, 7, 2, 25, 8, 1, 7, 5, 2, 8, 7, 25, 1, 2, 8, 7, 5, 1, 25, 3, 8, 2, 7, 1, 25, 8, 1, 5, 2, 7, 8, 25, 1, 2, 7, 8, 5, 2, 1, 8, 6, 7, 2, 1, 8, 25, 3, 8, 5, 7, 1, 2, 8, 25, 7, 7, 5, 1, 8, 2, 2, 1, 7, 25, 1, 8, 2, 6, 8, 3, 5, 7, 1, 2, 8, 25, 1, 7, 5, 1, 3, 7, 25, 8, 1, 3, 7, 5, 2, 8, 1, 7, 2, 25, 8, 1, 25, 7, 1, 8, 5, 2, 8, 6, 1, 7, 2, 25, 8, 5, 7, 1, 2, 1, 8, 3, 25, 7, 8, 5, 8, 8, 2, 2, 2, 1, 8, 25, 7, 2, 7, 2, 5, 1, 8, 25, 1, 8, 7, 2, 7, 6, 7, 5, 1, 3, 25, 7, 8, 7, 1, 2, 5, 8, 8, 5, 1, 1, 2, 25, 2, 7, 3, 25, 1, 7, 8, 2, 5, 7, 8, 25, 1, 7, 2, 1, 8, 5, 2, 7, 2], 'identity_hate': [8, 6, 2, 7, 1, 3, 5, 8, 7, 2, 6, 8, 1, 7, 2, 8, 6, 2, 7, 4, 3, 6, 1, 8, 5, 7, 2, 8, 6, 1, 7, 2, 3, 8, 6, 2, 7, 8, 1, 5, 7, 2, 6, 8, 3, 7, 2, 8, 2, 6, 1, 7, 8, 5, 8, 6, 2, 7, 1, 3, 8, 6, 7, 2, 2, 7, 8, 1, 6, 4, 3, 5, 7, 2, 8, 1, 6, 7, 2, 8, 1, 6, 7, 2, 8, 3, 8, 5, 7, 6, 8, 2, 7, 2, 1, 6, 7, 8, 1, 2, 6, 8, 3, 5, 7, 8, 2, 6, 1, 8, 7, 2, 3, 6, 5, 8, 7, 2, 8, 7, 

http://www.cs.cornell.edu/~alexn/papers/shotgun.icml04.revised.rev2.pdf

In [31]:
hill = pd.read_csv("hillclimb.csv")
hill.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.906870,0.345042,0.807456,0.095349,0.794653,0.420563
1,0000247867823ef7,0.026323,0.012885,0.013953,0.002785,0.016283,0.003830
2,00013b17ad220c46,0.029738,0.009850,0.013280,0.003224,0.012551,0.003344
3,00017563c3f7919a,0.019089,0.009829,0.009081,0.003597,0.008285,0.002841
4,00017695ad8997eb,0.030369,0.009837,0.013743,0.003243,0.012905,0.003249


In [5]:
opt_w

{'identity_hate': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0},
 'insult': {1: 0, 2: 0, 4: 0, 5: 0, 7: 0, 8: 0, 25: 0},
 'obscene': {1: 0, 2: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 25: 0},
 'severe_toxic': {1: 0, 2: 0, 3: 0, 5: 0, 7: 0, 8: 0, 25: 0},
 'threat': {1: 0, 2: 0, 3: 0, 5: 0, 6: 0, 7: 0, 8: 0, 25: 0},
 'toxic': {1: 0, 2: 0, 4: 0, 5: 0, 7: 0, 8: 0, 25: 0}}

In [8]:
best_score

{'identity_hate': 0.9913096169108873,
 'insult': 0.9898060621358835,
 'obscene': 0.9951883562665469,
 'severe_toxic': 0.992048945016528,
 'threat': 0.9940496107813258,
 'toxic': 0.9876047223608075}

In [9]:
best_ensemble

{'identity_hate': [8,
  6,
  2,
  7,
  1,
  3,
  5,
  8,
  7,
  2,
  6,
  8,
  1,
  7,
  2,
  8,
  6,
  2,
  7,
  4,
  3,
  6,
  1,
  8,
  5],
 'insult': [8,
  7,
  25,
  1,
  2,
  7,
  8,
  1,
  25,
  7,
  8,
  1,
  25,
  7,
  8,
  1,
  4,
  25,
  5,
  1,
  8,
  7,
  25,
  8,
  1],
 'obscene': [8,
  25,
  2,
  7,
  1,
  25,
  6,
  8,
  25,
  1,
  25,
  7,
  5,
  8,
  2,
  25,
  1,
  25,
  4,
  7,
  8,
  25,
  1,
  25,
  25],
 'severe_toxic': [8,
  2,
  25,
  7,
  1,
  5,
  8,
  3,
  1,
  7,
  2,
  25,
  8,
  1,
  7,
  5,
  2,
  8,
  7,
  25,
  1,
  2,
  8,
  7,
  5],
 'threat': [1,
  25,
  1,
  3,
  2,
  1,
  7,
  1,
  5,
  8,
  1,
  6,
  1,
  2,
  5,
  1,
  3,
  1,
  8,
  5,
  7,
  1,
  2,
  5,
  1],
 'toxic': [8,
  25,
  1,
  7,
  25,
  2,
  8,
  25,
  1,
  25,
  7,
  8,
  5,
  1,
  25,
  2,
  25,
  8,
  7,
  1,
  4,
  25,
  25,
  1,
  8]}